In [ ]:
# creating spectrograms from all the files, and saving split labelled versions to disk ready for machine learning

%load_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt

import os
import sys
import cPickle as pickle

from data_helpers import load_annotations

where_to_save = '/media/michael/Seagate/engage/alison_data/golden_set/extracted/annotations/'
base_path = '/media/michael/Seagate/engage/alison_data/golden_set/labels/Golden/'